In [1]:
import glob
import xarray as xr
import netCDF4
import h5netcdf
import scipy
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [2]:
file_path='../Data/'
data_list_in=glob.glob(file_path+'ds_in*')
data_list_out=glob.glob(file_path+'ds_out*')
data_list_in.sort()
data_list_out.sort()
print(data_list_in)
print(data_list_out)

['../Data/ds_in.csv', '../Data/ds_inBering.csv', '../Data/ds_inBering30day10.csv', '../Data/ds_inBering30day20.csv', '../Data/ds_inSH.csv', '../Data/ds_inSH30day10.csv', '../Data/ds_inSH30day20.csv', '../Data/ds_inSHWeddell30day10.csv', '../Data/ds_inSHWeddell30day20.csv', '../Data/ds_inWeddell.csv', '../Data/ds_in_30day10.csv', '../Data/ds_in_30day20.csv']
['../Data/ds_out.csv', '../Data/ds_outBering.csv', '../Data/ds_outBering30day10.csv', '../Data/ds_outBering30day20.csv', '../Data/ds_outSH.csv', '../Data/ds_outSH30day10.csv', '../Data/ds_outSH30day20.csv', '../Data/ds_outSHWedell30day10.csv', '../Data/ds_outSHWedell30day20.csv', '../Data/ds_outWeddell.csv', '../Data/ds_out_30day10.csv', '../Data/ds_out_30day20.csv']


In [3]:
data_arrays_in = [np.loadtxt(file, delimiter=',') for file in data_list_in]
data_arrays_out = [np.loadtxt(file, delimiter=',') for file in data_list_out]

# Print the shapes of the arrays to verify
for i, array in enumerate(data_arrays_in):
    print(f"Shape of data_arrays_in[{i}]: {array.shape}")

for i, array in enumerate(data_arrays_out):
    print(f"Shape of data_arrays_out[{i}]: {array.shape}")

Shape of data_arrays_in[0]: (45693, 13)
Shape of data_arrays_in[1]: (41227, 13)
Shape of data_arrays_in[2]: (30775, 13)
Shape of data_arrays_in[3]: (56466, 13)
Shape of data_arrays_in[4]: (44050, 13)
Shape of data_arrays_in[5]: (41869, 13)
Shape of data_arrays_in[6]: (3, 13)
Shape of data_arrays_in[7]: (16273, 13)
Shape of data_arrays_in[8]: (371, 13)
Shape of data_arrays_in[9]: (15356, 13)
Shape of data_arrays_in[10]: (86622, 13)
Shape of data_arrays_in[11]: (71015, 13)
Shape of data_arrays_out[0]: (45693, 12)
Shape of data_arrays_out[1]: (41227, 12)
Shape of data_arrays_out[2]: (30775, 12)
Shape of data_arrays_out[3]: (56466, 12)
Shape of data_arrays_out[4]: (44050, 12)
Shape of data_arrays_out[5]: (41869, 12)
Shape of data_arrays_out[6]: (3, 12)
Shape of data_arrays_out[7]: (16273, 12)
Shape of data_arrays_out[8]: (371, 12)
Shape of data_arrays_out[9]: (15356, 12)
Shape of data_arrays_out[10]: (86622, 12)
Shape of data_arrays_out[11]: (71015, 12)


In [4]:
# Reshape each array in data_arrays_in to the specified shape and concatenate them
reshaped_data_arrays_in = np.concatenate([array.reshape(-1, 13) for array in data_arrays_in])

# Print the shape of the reshaped array to verify
print(f"Shape of reshaped_data_arrays_in: {reshaped_data_arrays_in.shape}")

Shape of reshaped_data_arrays_in: (449720, 13)


In [5]:
# Reshape each array in data_arrays_out to the specified shape and concatenate them
reshaped_data_arrays_out = np.concatenate([array.reshape(-1, 12) for array in data_arrays_out])

# Print the shape of the reshaped array to verify
print(f"Shape of reshaped_data_arrays_out: {reshaped_data_arrays_out.shape}")

Shape of reshaped_data_arrays_out: (449720, 12)


In [6]:
# Clean data_arrays_in and data_arrays_out
cleaned_data_arrays_in = []
cleaned_data_arrays_out = []

for i in range(reshaped_data_arrays_in.shape[0]):
        if not np.sum(reshaped_data_arrays_out[i])==0:
            cleaned_data_arrays_in.append(reshaped_data_arrays_in[i])
            cleaned_data_arrays_out.append(reshaped_data_arrays_out[i])

# Convert lists back to numpy arrays if needed
cleaned_data_arrays_in = np.array(cleaned_data_arrays_in)
cleaned_data_arrays_out = np.array(cleaned_data_arrays_out)

# Print the shapes of the cleaned arrays to verify
print(f"Shape of cleaned_data_arrays_in: {cleaned_data_arrays_in.shape}")
print(f"Shape of cleaned_data_arrays_out: {cleaned_data_arrays_out.shape}")

Shape of cleaned_data_arrays_in: (422480, 13)
Shape of cleaned_data_arrays_out: (422480, 12)


In [76]:
# Calculate the orders of magnitude difference
def calculate_orders_of_magnitude_difference(predictions, actuals):
    differences = torch.abs(predictions - actuals)
    orders_of_magnitude_diff = torch.log10(differences + 1e-10) - torch.log10(torch.abs(actuals) + 1e-10)
    return torch.mean(torch.abs(orders_of_magnitude_diff))

In [77]:
import torch
from sklearn.model_selection import train_test_split

import torch.nn as nn
import torch.optim as optim

# Convert data to PyTorch tensors
X_train, X_test, y_train, y_test = train_test_split(cleaned_data_arrays_in, cleaned_data_arrays_out, test_size=0.2, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Define the neural network architecture
class PhysicsInformedNN(nn.Module):
    def __init__(self):
        super(PhysicsInformedNN, self).__init__()
        self.hidden1 = nn.Linear(X_train.shape[1], 256)
        self.hidden2 = nn.Linear(256, 128)
        self.hidden3 = nn.Linear(128, 64)
        self.output = nn.Linear(64, y_train_tensor.shape[1])
    
    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = torch.relu(self.hidden3(x))
        x = self.output(x)
        return x

# Initialize the model, loss function, and optimizer
model = PhysicsInformedNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Custom loss function to enforce the sum of outputs to be zero
def custom_loss(output, target):
    mse_loss = criterion(output, target)

    sign_penalty = 0
    for i in range(output.shape[1]):
        sign_penalty += torch.mean(torch.relu(-output[:, i] * torch.sign(target[:, i])))
    sign_penalty = sign_penalty / output.shape[1]
    
    sign_loss=sign_penalty**2
    sum_constraint = torch.sum(output, dim=1)
    sum_loss = torch.mean(sum_constraint ** 2)
    return [mse_loss, sum_loss,sign_loss]

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = custom_loss(outputs, y_train_tensor)
    loss_sum= loss[0]+loss[1]+1000*loss[2] 
    loss_sum.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], MSE Loss: {loss[0].item():.6f} Sum loss: {loss[1].item():.6f} Sign Loss: {loss[2].item():.6f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    mse = criterion(predictions, y_test_tensor)
    print(f'Test Mean Squared Error: {mse.item():.20f}')

Epoch [100/1000], MSE Loss: 0.000569 Sum loss: 0.000003 Sign Loss: 0.000000
Epoch [200/1000], MSE Loss: 0.000016 Sum loss: 0.000000 Sign Loss: 0.000000
Epoch [300/1000], MSE Loss: 0.000013 Sum loss: 0.000000 Sign Loss: 0.000000
Epoch [400/1000], MSE Loss: 0.000013 Sum loss: 0.000000 Sign Loss: 0.000000
Epoch [500/1000], MSE Loss: 0.000013 Sum loss: 0.000000 Sign Loss: 0.000000
Epoch [600/1000], MSE Loss: 0.000012 Sum loss: 0.000000 Sign Loss: 0.000000
Epoch [700/1000], MSE Loss: 0.000011 Sum loss: 0.000000 Sign Loss: 0.000000
Epoch [800/1000], MSE Loss: 0.000010 Sum loss: 0.000000 Sign Loss: 0.000000
Epoch [900/1000], MSE Loss: 0.000010 Sum loss: 0.000000 Sign Loss: 0.000000
Epoch [1000/1000], MSE Loss: 0.000009 Sum loss: 0.000000 Sign Loss: 0.000000
Test Mean Squared Error: 0.00000880298011907144


In [78]:
ds_in=cleaned_data_arrays_in
ds_out=cleaned_data_arrays_out

In [79]:
k=25009
print(model(torch.tensor(ds_in[k], dtype=torch.float32)))
print(sum(model(torch.tensor(ds_in[k], dtype=torch.float32))))
print(ds_in[k])
print(ds_out[k])

tensor([ 5.3028e-04,  1.2299e-03,  5.0677e-04,  6.3368e-04,  3.0687e-04,
         2.8238e-04,  1.7975e-05, -3.2791e-04, -6.0621e-04, -6.1639e-04,
        -1.3885e-03, -4.0108e-04], grad_fn=<ViewBackward0>)
tensor(0.0002, grad_fn=<AddBackward0>)
[9.86153960e-01 7.37698516e-03 6.60287275e-04 9.57532786e-04
 1.49875123e-03 2.31453776e-03 3.02756933e-04 4.31457993e-05
 9.21014216e-05 1.35843962e-04 2.43077782e-04 2.21020615e-04
 3.55377293e-02]
[ 1.62720680e-05  4.52380627e-05  9.28282971e-05  1.66275189e-04
  2.75537139e-04  4.32153698e-04 -2.93110381e-04 -4.31457993e-05
 -9.21014216e-05 -1.35843962e-04 -2.43077782e-04 -2.21020615e-04]


In [80]:
negative_sign_differences = 0
#ds_in_combine, ds_out_combine
for i in range(len(ds_in)):
    predicted_output = model(torch.tensor(ds_in[i], dtype=torch.float32))
    actual_output = torch.tensor(ds_out[i])
    
    # Compare the signs of the predicted and actual outputs
    predicted_signs = torch.sign(predicted_output)
    actual_signs = torch.sign(actual_output)
    
    # Count the number of different signs
    negative_sign_differences += torch.sum(predicted_signs != actual_signs).item()

print(f'Total number of incorrect different negative signs: {negative_sign_differences} As a %: {100*negative_sign_differences/(len(ds_in)*len(ds_out[0]))}')

Total number of incorrect different negative signs: 202102 As a %: 5.708689663899632


In [81]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor)
    avg_orders_of_magnitude_diff = calculate_orders_of_magnitude_difference(test_predictions, y_test_tensor)
    print(f'Average Orders of Magnitude Difference: {avg_orders_of_magnitude_diff.item():.6f}')

Average Orders of Magnitude Difference: 0.912179


In [82]:
# Calculate the average of each model run
averages = 0

for i in range(len(ds_in)):
    predicted_output = model(torch.tensor(ds_in[i], dtype=torch.float32))
    average_output = torch.sum(predicted_output).item()
    averages+=average_output

# Print the averages
averages=averages/len(ds_in)
print(f"Average output for model run {i}: {averages:.20f}")

Average output for model run 295020: 0.00003892220030844522


In [84]:
import pandas as pd

# Extract weights from the model
hidden1_weights = model.hidden1.weight.detach().numpy()
hidden2_weights = model.hidden2.weight.detach().numpy()
hidden3_weights = model.hidden3.weight.detach().numpy()

# Create a DataFrame for each layer's weights
df_hidden1 = pd.DataFrame(hidden1_weights)
df_hidden2 = pd.DataFrame(hidden2_weights)
df_hidden3 = pd.DataFrame(hidden3_weights)

# Save each DataFrame to a CSV file
df_hidden1.to_csv('hidden1_weights.csv', index=False, header=False)
df_hidden2.to_csv('hidden2_weights.csv', index=False, header=False)
df_hidden3.to_csv('hidden3_weights.csv', index=False, header=False)

print("Model weights saved to CSV files.")

Model weights saved to CSV files.


In [40]:
#Let's just try to classify sign
cleaned_data_arrays_out_sign=[]
for i in range(len(cleaned_data_arrays_out)):
    cleaned_data_arrays_out_sign.append(np.sign(cleaned_data_arrays_out[i]))
cleaned_data_arrays_out_sign=np.array(cleaned_data_arrays_out_sign)
print(cleaned_data_arrays_out_sign.shape)
print(cleaned_data_arrays_in.shape)

(131674, 12)
(131674, 13)


In [51]:
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(cleaned_data_arrays_in, cleaned_data_arrays_out_sign, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # Use `long` for integer labels
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

print(f"X_train_tensor shape: {X_train_tensor.shape}")
print(f"y_train_tensor shape: {y_train_tensor.shape}")
# Define the model
class BinaryClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(X_test.shape[1], 256)  # First layer with 64 units
        self.fc2 = nn.Linear(256, 128)  # Second layer with 32 units
        self.fc3 = nn.Linear(128, 64)  # Output layer
        self.fc4 = nn.Linear(64, y_test.shape[1],)  # Output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

# Define input and output sizes based on the tensor shapes
input_size = 13  # X has 13 features
output_size = 12  # y has 12 labels

# Instantiate the model, define loss and optimizer
model = BinaryClassifier(input_size, output_size)
criterion = nn.BCELoss()  # Binary cross-entropy for multi-label binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert y_train_tensor to range [0, 1] (optional)
y_train_tensor = (y_train_tensor + 1) / 2

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Convert outputs back to -1 or 1 after prediction if needed
final_outputs = (outputs * 2) - 1


X_train_tensor shape: torch.Size([105339, 13])
y_train_tensor shape: torch.Size([105339, 12])
Epoch [1/10], Loss: 0.6973
Epoch [2/10], Loss: 0.6917
Epoch [3/10], Loss: 0.6860
Epoch [4/10], Loss: 0.6798
Epoch [5/10], Loss: 0.6731
Epoch [6/10], Loss: 0.6658
Epoch [7/10], Loss: 0.6580
Epoch [8/10], Loss: 0.6491
Epoch [9/10], Loss: 0.6394
Epoch [10/10], Loss: 0.6285


In [ ]:
ds_in=cleaned_data_arrays_in
ds_out=cleaned_data_arrays_out_sign

In [52]:
negative_sign_differences = 0
#ds_in_combine, ds_out_combine
for i in range(len(ds_in)):
    predicted_output = model(torch.tensor(ds_in[i], dtype=torch.float32))
    actual_output = torch.tensor(ds_out[i])
    
    # Compare the signs of the predicted and actual outputs
    predicted_signs = torch.sign(predicted_output)
    actual_signs = torch.sign(actual_output)
    
    # Count the number of different signs
    negative_sign_differences += torch.sum(predicted_signs != actual_signs).item()

print(f'Total number of different negative signs: {negative_sign_differences} As a %: {100*negative_sign_differences/(len(ds_in)*len(ds_in[0]))}')

Total number of different negative signs: 382523 As a %: 22.346739792097267
